In [ ]:
!pip install wurlitzer
!pip install tensorflow 
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, InputLayer, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
import plotly.express as px
import matplotlib.pyplot as plt

#import os  #when import os firstly make deadlock 
#/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.pid, fd = os.forkpty()
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#first check availability of internet otherwise will not run properlly 
import requests

try:
    response = requests.get("https://www.google.com/")
    response.raise_for_status()  # Raise an exception for failed requests
    print("Internet connection seems to be working!")
except requests.exceptions.RequestException as e:
    print("Error: Could not connect to the internet.", e)

In [ ]:
import os  #when import os firstly make deadlock 
path = "/kaggle/input/tomatoleaf/tomato"
os.listdir(path)

In [ ]:
train_path = os.path.join(path, "train")
print(os.listdir(train_path))
print("*"*100)
test_path = os.path.join(path, "val")
print(os.listdir(test_path))

In [ ]:
from glob import glob
folders = glob("/kaggle/input/tomatoleaf/tomato/train/*")
folders

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(plt.imread("/kaggle/input/tomatoleaf/tomato/train/Tomato___Bacterial_spot/05777829-6912-44bb-bcef-901bc5190584___GCREC_Bact.Sp 6150.JPG"))
plt.axis('off')  # Turn off axis
plt.title("Bacterial Spot")

In [ ]:
import tensorflow 
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image

In [ ]:
SIZE = [128, 128,3]
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
cnn=Sequential()
# Create a new Sequential model
cnn = models.Sequential()
# First block
cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Second block
cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Third block
cnn.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Fourth block
cnn.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Fifth block
cnn.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Flatten the output of the convolutional base
cnn.add(layers.Flatten())

# Fully connected layers
cnn.add(layers.Dense(1024, activation='relu'))
cnn.add(layers.Dropout(0.5))
cnn.add(layers.Dense(512, activation='relu'))
cnn.add(layers.Dropout(0.5))

# Output layer for classification tasks
# Set the number of units to the number of classes (e.g., 10 for 10 classes)
cnn.add(layers.Dense(10, activation='softmax'))

# Compile the model
#cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])






In [ ]:
cnn.summary()

In [ ]:
train_datagen_vg19 = ImageDataGenerator(rescale=1./255)

test_datagen_vg19 = ImageDataGenerator(rescale=1./255)

In [ ]:
trainning_set_vg19 = train_datagen_vg19.flow_from_directory(train_path,
                                                 target_size=(128, 128),
                                                 batch_size=30,
                                                 class_mode="categorical", shuffle=True)

In [ ]:
testing_set_vg19 = test_datagen_vg19.flow_from_directory(test_path,
                                                 target_size=(128, 128),
                                                 batch_size=30,
                                                 class_mode="categorical", shuffle=False)

In [ ]:
from tensorflow.keras.optimizers import Adam

# Specify the desired learning rate (e.g., 0.001)
learning_rate = 1e-4  # You can adjust the value as needed

# Create an instance of the Adam optimizer with the specified learning rate
adam_optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the desired optimizer, loss function, and metrics
cnn.compile(loss="categorical_crossentropy",
                metrics=["accuracy"],
                optimizer=adam_optimizer)

In [ ]:
# Define your callbacks
callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5 , restore_best_weights=True)

# Train the model using the fit method
r_vg19 = cnn.fit(trainning_set_vg19,
                       validation_data=testing_set_vg19,
                       epochs=50,
                       callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt
accuracy = r_vg19.history['accuracy']
val_accuracy = r_vg19.history['val_accuracy']
loss = r_vg19.history['loss']
val_loss = r_vg19.history['val_loss']
epochs = range(len(accuracy))
plt.title("model")
plt.plot(epochs, accuracy, "b", label="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, "b", label="trainning loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.legend()
plt.show()
#cnn.save('Leaf_Life_model')


In [ ]:
y_pred = cnn.predict(testing_set_vg19)
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
test_img = plt.imread("/kaggle/input/tomatoleaf/tomato/train/Tomato___Bacterial_spot/00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG")
plt.imshow(test_img)
cnn.save("vgg_19tl.model")

In [ ]:
#import cv2
#import tensorflow as tf
def prepare(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    new_array = cv2.resize(img_array, (128, 128))
    return new_array.reshape(-1, 128, 128, 3)

model = tf.keras.models.load_model("vgg_19tl.model")

In [ ]:
class_dict = trainning_set_vg19.class_indices
class_dict

In [ ]:
def prediction_cls(prediction):
    for key, clss in class_dict.items():
        if np.argmax(prediction) == clss:
            return key

In [ ]:
import cv2
prediction = model.predict([prepare("/kaggle/input/tomatoleaf/tomato/val/Tomato___healthy/1b966d62-d5c3-4a67-ba36-b67d873ccfdf___GH_HL Leaf 440.JPG")])
prediction_cls(prediction)

In [ ]:
prediction = model.predict([prepare("/kaggle/input/tomatoleaf/tomato/val/Tomato___Leaf_Mold/0ba88812-fa4f-4602-bcec-c03cd7d0ba2b___Crnl_L.Mold 6990.JPG")])

prediction_cls(prediction)

In [ ]:
# Specify the path to save the model
model_path = '/kaggle/working/smartLeaf.h5'

# Save the model
cnn.save(model_path)


In [ ]:
#import tensorflow as tf
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#tflite_model = converter.convert()
#tflite_model_save_path = "/kaggle/working/model.tflite"

# Save the TensorFlow Lite model to the specified path
#with open(tflite_model_save_path, "wb") as f:
#    f.write(tflite_model)